In [1]:
import sys
sys.path.append("/proj/gaia-climate/team/kirill/gaia-surrogate")

In [2]:
from gaia.models import FcnBaseline

### memory footprint

In [26]:
ins  = 164
outs = 54
def get_memory_footprint(l,h, b):
    model = FcnBaseline(input_size=ins, num_layers=l, hidden_size=h,output_size=outs)
    trainable_params = sum([p.numel() for p in model.parameters() if p.requires_grad])
    non_trainable_params = sum([p.numel() for p in model.parameters() if p.requires_grad])
    activations = l*(4*h)
    grads = trainable_params
    optimizer_states = grads*2
    
    p = trainable_params + non_trainable_params + grads + optimizer_states
    return (p+4*activations*b)/(10**9)

In [31]:
import pandas as pd

In [46]:
num_layers = [3,5,7,14,21]
hidden_size = [128,256,512,1024,2048]

df = []

for l in num_layers:
    for h in hidden_size:
        for b in [2,4,8,16,24]:
            df.append((l,h,b,get_memory_footprint(l, h, b*96*144)))

In [47]:
df = pd.DataFrame(df,columns=["layers","size","batch","memory"]).sort_values("memory")

In [51]:
df.query("memory<=20").sort_values("batch", ascending = False).drop_duplicates(["layers","size"]).sort_values(["layers","size"]).reset_index(drop = True).to_csv("memory_size.csv",index = False)

In [6]:
ins  = 164
outs = 54
def compute_memory_footprint(l,h):
    num_params_diff_params =  (l-2)*(h**2 + h) + h*(64+1) + h*(164+1) + (h*2)*(l-1)
    num_non_diff_params = ins

In [7]:
compute_num_params(21,2048)/compute_num_params(7,512)

56.34400863620007

In [16]:
from math import log,log10, log2
log2(56)

5.807354922057604

In [52]:
memory_size = pd.read_csv("memory_size.csv").set_index(["layers","size"])


In [53]:
memory_size.loc[(3,512)]

batch     24.000000
memory     8.155611
Name: (3, 512), dtype: float64

### get data

In [61]:
from gaia.models import TrainingModel
from gaia.training import get_checkpoint_file, load_hparams_file
import glob,os

In [64]:
model_dirs = [m for m in glob.glob("lightning_logs/*")]#if os.path.exists(os.path.join(m,"test_results.json"))]
data = []

fields = ["hidden_size","num_layers","model_type"]
for m in model_dirs:
    
    model = TrainingModel.load_from_checkpoint(get_checkpoint_file(m), map_location="cpu")
    
    num_params = sum([p.numel() for p in model.parameters()])
    
#     params = load_hparams_file(m)
#     if params:
#         test_file = os.path.join(m,"test_results.json")
#         test_results = pd.read_json(test_file, orient = "records").iloc[0]
#         for f in fields:
#             test_results[f] = params["model_config"][f]
        
#         # test_results["ignore_input_variables"] = params.get("ignore_input_variables")
#         # test_results["memory_variables"] = params.get("memory_variables")
#         data.append(test_results.to_frame().T)
        
# data = pd.concat(data,ignore_index=True)

2023-01-25 12:00:23,011 - gaia.gaia.models - WARNING - no stats provided, assuming will be loaded later, initializing randomly
2023-01-25 12:00:23,715 - gaia.gaia.models - INFO - using output weights [0.5500000715255737, 0.5500000715255737, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.351999968290329, 0.3879999816417694, 0.42399996519088745, 0.4599999785423279, 0.4959999918937683, 0.531999945640564, 0.5679999589920044, 0.6039999723434448, 0.6399999856948853, 0.6759999990463257, 0.7120000123977661, 0.7480000257492065, 0.784000039100647, 0.8199999928474426, 0.8560000061988831, 0.8920000195503235, 0.9279999732971191, 0.9639999866485596, 1.0, 0.10000000149011612, 0.13600000739097595, 0.171999990940094, 0.20800000429153442, 0.24399998784065247, 0.2800000011920929, 0.31599998474121094, 0.351999968290329, 0.3879999816417694, 0.42399996519088745, 0.4599999785423279, 0.4959999918937683, 0.531999945640564, 0.5679999589920044, 0.6039999723434448, 0.6399999856948853, 0.6759999990463257, 0.7120000123977661

In [65]:
model_dirs

['lightning_logs/version_0',
 'lightning_logs/version_1',
 'lightning_logs/version_2',
 'lightning_logs/version_3',
 'lightning_logs/version_4',
 'lightning_logs/version_5',
 'lightning_logs/version_6',
 'lightning_logs/version_7',
 'lightning_logs/version_8',
 'lightning_logs/version_9',
 'lightning_logs/version_10',
 'lightning_logs/version_11',
 'lightning_logs/version_12',
 'lightning_logs/version_13',
 'lightning_logs/version_14',
 'lightning_logs/version_15',
 'lightning_logs/version_16',
 'lightning_logs/version_17',
 'lightning_logs/version_18',
 'lightning_logs/version_19',
 'lightning_logs/version_20']